# DB.nomics International Real GDP Quarterly Growth

## Python Setup

In [ ]:
# system imports
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
import time
import requests
from datetime import datetime, timedelta
import json

In [ ]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
# local imports
import mgplot as mg

In [ ]:
# plotting setup
CHART_DIR = "./CHARTS/DBNOMICS_COMPLETE/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()
SHOW = False
SOURCE = "DB.nomics (IMF, Eurostat, OECD)"

## Complete Country Coverage & Working Sources

In [ ]:
# DB.nomics API Configuration
DBNOMICS_BASE_URL = "https://api.db.nomics.world/v22"

# Complete OECD + G20 countries with WORKING DB.nomics sources
# Priority: Eurostat (most current) -> IMF -> OECD
country_sources_complete = {
    # === G7 Countries ===
    "USA": {
        "name": "United States",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.US.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "JPN": {
        "name": "Japan",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.JP.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "DEU": {
        "name": "Germany",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.DE", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.DE.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "GBR": {
        "name": "United Kingdom",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.GB.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "FRA": {
        "name": "France",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.FR", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.FR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "ITA": {
        "name": "Italy",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.IT", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.IT.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "CAN": {
        "name": "Canada",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CA.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    
    # === Other Major G20 Economies ===
    "CHN": {
        "name": "China",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CN.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "IND": {
        "name": "India",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.IN.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "BRA": {
        "name": "Brazil",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.BR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "RUS": {
        "name": "Russia",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.RU.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "AUS": {
        "name": "Australia",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.AU.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "KOR": {
        "name": "South Korea",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.KR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "MEX": {
        "name": "Mexico",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.MX.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "ARG": {
        "name": "Argentina",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.AR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "IDN": {
        "name": "Indonesia",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.ID.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "SAU": {
        "name": "Saudi Arabia",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.SA.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "ZAF": {
        "name": "South Africa",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.ZA.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "TUR": {
        "name": "Turkey",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.TR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "SGP": {
        "name": "Singapore",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.SG.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    
    # === OECD European Countries ===
    "ESP": {
        "name": "Spain",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.ES", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.ES.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "NLD": {
        "name": "Netherlands",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.NL", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.NL.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "BEL": {
        "name": "Belgium",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.BE", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.BE.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "SWE": {
        "name": "Sweden",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.SE", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.SE.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "AUT": {
        "name": "Austria",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.AT", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.AT.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "DNK": {
        "name": "Denmark",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.DK", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.DK.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "FIN": {
        "name": "Finland",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.FI", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.FI.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "NOR": {
        "name": "Norway",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.NO.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "CHE": {
        "name": "Switzerland",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CH.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "POL": {
        "name": "Poland",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.PL", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.PL.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "CZE": {
        "name": "Czech Republic",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.CZ", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CZ.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "HUN": {
        "name": "Hungary",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.HU", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.HU.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "SVK": {
        "name": "Slovakia",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.SK", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.SK.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "SVN": {
        "name": "Slovenia",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.SI", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.SI.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "EST": {
        "name": "Estonia",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.EE", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.EE.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "LVA": {
        "name": "Latvia",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.LV", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.LV.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "LTU": {
        "name": "Lithuania",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.LT", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.LT.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "LUX": {
        "name": "Luxembourg",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.LU", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.LU.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "PRT": {
        "name": "Portugal",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.PT", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.PT.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "GRC": {
        "name": "Greece",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.EL", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.GR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "IRL": {
        "name": "Ireland",
        "sources": [
            {"provider": "Eurostat", "dataset": "namq_10_gdp", "series": "Q.CLV10_MEUR.SCA.B1GQ.IE", "type": "levels"},
            {"provider": "IMF", "dataset": "IFS", "series": "Q.IE.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "ISL": {
        "name": "Iceland",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.IS.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "ISR": {
        "name": "Israel",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.IL.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "NZL": {
        "name": "New Zealand",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.NZ.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "CHL": {
        "name": "Chile",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CL.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "COL": {
        "name": "Colombia",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CO.NGDP_R_SA_XDC", "type": "levels"},
        ]
    },
    "CRI": {
        "name": "Costa Rica",
        "sources": [
            {"provider": "IMF", "dataset": "IFS", "series": "Q.CR.NGDP_R_SA_XDC", "type": "levels"},
        ]
    }
}

## Complete Data Retrieval Functions

In [ ]:
def get_gdp_data_complete(
    country_code: str, 
    country_info: Dict[str, Any], 
    start_date: str = "2000-01-01"
) -> Optional[Tuple[pd.Series, str]]:
    """Get real GDP data for a single country using working DB.nomics sources.
    
    Args:
        country_code: ISO 3-letter country code
        country_info: Dictionary with country name and sources
        start_date: Start date for data retrieval
    
    Returns:
        Tuple of (pandas Series with quarterly GDP growth rates, source_provider) or None
    """
    
    country_name = country_info["name"]
    sources = country_info["sources"]
    
    # Try each source in priority order
    for i, source in enumerate(sources, 1):
        provider = source["provider"]
        dataset = source["dataset"]
        series_code = source["series"]
        data_type = source["type"]
        
        try:
            # Build DB.nomics URL
            url = f"{DBNOMICS_BASE_URL}/series/{provider}/{dataset}/{series_code}"
            params = {"observations": "1"}
            
            response = requests.get(url, params=params, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                docs = data.get('series', {}).get('docs', [])
                
                if docs and len(docs) > 0:
                    doc = docs[0]
                    periods = doc.get('period', [])
                    values = doc.get('value', [])
                    
                    if periods and values and len(periods) >= 10:
                        # Filter out None values
                        valid_data = [(p, v) for p, v in zip(periods, values) if v is not None]
                        
                        if len(valid_data) >= 10:
                            periods_clean, values_clean = zip(*valid_data)
                            
                            # Create pandas Series
                            series = pd.Series(
                                data=values_clean,
                                index=pd.PeriodIndex(periods_clean, freq='Q'),
                                name=country_name
                            )
                            
                            # Filter to data from start_date onwards
                            start_year = int(start_date.split('-')[0])
                            series = series[series.index.year >= start_year]
                            
                            if len(series) >= 10:
                                # Calculate quarterly growth rates from levels
                                if data_type == "levels":
                                    growth = series.pct_change(periods=1) * 100
                                    growth = growth.dropna()
                                else:
                                    # Already growth rates
                                    growth = series
                                
                                if len(growth) >= 8:
                                    print(f"✅ {len(growth)} quarters ({provider})")
                                    return growth, provider
                                    
        except Exception as e:
            continue
    
    return None

In [ ]:
def collect_all_gdp_data_complete(countries: Dict[str, Dict], start_date: str = "2000-01-01") -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
    """Collect GDP growth data for all countries using working DB.nomics sources.
    
    Args:
        countries: Dictionary mapping country codes to country info
        start_date: Start date for data retrieval
    
    Returns:
        Tuple of (DataFrame with quarterly GDP growth rates, source_summary dict)
    """
    
    gdp_data = {}
    successful_fetches = []
    failed_fetches = []
    source_summary = {}
    
    print(f"Collecting real GDP data for {len(countries)} countries from DB.nomics...")
    print("Using WORKING sources: IMF, Eurostat, OECD")
    print("="*80)
    
    for i, (country_code, country_info) in enumerate(countries.items(), 1):
        country_name = country_info["name"]
        print(f"[{i:2d}/{len(countries)}] Fetching {country_name:<20} ({country_code})...", end=" ")
        
        # Try to get data for this country
        result = get_gdp_data_complete(country_code, country_info, start_date)
        
        if result is not None:
            series, provider = result
            gdp_data[country_name] = series
            successful_fetches.append((country_name, country_code, provider))
            
            # Track which source was used
            if provider not in source_summary:
                source_summary[provider] = []
            source_summary[provider].append(country_name)
        else:
            print("❌ No data found")
            failed_fetches.append((country_name, country_code))
        
        time.sleep(0.2)  # Be nice to DB.nomics API
    
    # Print comprehensive summary
    print("\n" + "="*80)
    print("DB.NOMICS GDP DATA COLLECTION SUMMARY")
    print("="*80)
    print(f"✅ Successfully fetched: {len(successful_fetches)} countries")
    print(f"❌ Failed to fetch: {len(failed_fetches)} countries")
    
    if successful_fetches:
        print("\n✅ SUCCESSFUL COUNTRIES:")
        for country, code, provider in successful_fetches:
            print(f"   {country:<25} ({code}) - {provider}")
    
    if failed_fetches:
        print("\n❌ FAILED COUNTRIES:")
        for country, code in failed_fetches:
            print(f"   {country:<25} ({code})")
    
    # Print source breakdown
    if source_summary:
        print("\n📊 ACTUAL DATA SOURCES USED:")
        for provider, country_list in source_summary.items():
            if country_list:  
                print(f"   {provider}: {len(country_list)} countries")
                if len(country_list) <= 5:
                    print(f"     {', '.join(country_list)}")
                else:
                    print(f"     {', '.join(country_list[:5])}... and {len(country_list)-5} more")
    
    if not gdp_data:
        print("\n⚠️ No data retrieved!")
        return pd.DataFrame(), {}
    
    df = pd.DataFrame(gdp_data)
    return df.dropna(how="all"), source_summary

In [ ]:
def analyze_data_currency(df: pd.DataFrame) -> None:
    """Analyze how current the collected data is."""
    
    if df.empty:
        print("❌ No data to analyze")
        return
    
    print(f"\n📊 DB.NOMICS GDP DATA ANALYSIS")
    print(f"   Total countries: {len(df.columns)}")
    print(f"   Time periods: {len(df)}")
    print(f"   Date range: {df.index[0]} to {df.index[-1]}")
    
    # Analyze recency by country
    latest_data = df.apply(lambda x: x.last_valid_index())
    
    # Group countries by data recency
    current_year = datetime.now().year
    current_quarter = (datetime.now().month - 1) // 3 + 1
    
    very_recent = []  # Q2 2024 or later
    recent = []       # Q4 2023 - Q1 2024  
    lagged = []       # Before Q4 2023
    
    for country, last_date in latest_data.items():
        if last_date is None:
            continue
            
        if last_date.year == 2024 and last_date.quarter >= 2:
            very_recent.append(country)
        elif (last_date.year == 2024 and last_date.quarter >= 1) or \
             (last_date.year == 2023 and last_date.quarter >= 4):
            recent.append(country)
        else:
            lagged.append(country)
    
    if very_recent:
        print(f"\n🎉 VERY RECENT DATA (Q2 2024+): {len(very_recent)} countries")
        print(f"   {', '.join(very_recent[:10])}{'...' if len(very_recent) > 10 else ''}")
    
    if recent:
        print(f"\n🔶 RECENT DATA (Q4 2023 - Q1 2024): {len(recent)} countries")  
        print(f"   {', '.join(recent[:8])}{'...' if len(recent) > 8 else ''}")
    
    if lagged:
        print(f"\n⚠️  LAGGED DATA (Before Q4 2023): {len(lagged)} countries")
        print(f"   {', '.join(lagged[:8])}{'...' if len(lagged) > 8 else ''}")
    
    # Show latest period coverage
    final_row = df.iloc[-1]
    available_count = final_row.notna().sum()
    coverage_pct = (available_count / len(df.columns)) * 100
    
    print(f"\n📈 LATEST PERIOD COVERAGE: {final_row.name}")
    print(f"   Countries with data: {available_count}/{len(df.columns)} ({coverage_pct:.1f}%)")
    
    if available_count > 0:
        available_countries = df.columns[final_row.notna()].tolist()
        print(f"   With latest data: {', '.join(available_countries[:8])}{'...' if len(available_countries) > 8 else ''}")

## Complete Data Collection

In [ ]:
# Collect GDP data from all countries using working sources
print("🌍 COLLECTING COMPLETE GDP DATA FROM DB.NOMICS")
print("Using realistic international sources that actually work")
print("Focus: Real GDP (chain volume measures) seasonally adjusted")
print()

# Get GDP data for all countries
gdp_complete, sources_used_complete = collect_all_gdp_data_complete(country_sources_complete, start_date="2000-01-01")

if not gdp_complete.empty:
    analyze_data_currency(gdp_complete)
else:
    print("\n⚠️ No data retrieved from DB.nomics.")

In [ ]:
# Chart groupings for analysis
chart_groups = {
    "g7": ["United States", "Japan", "Germany", "United Kingdom", "France", "Italy", "Canada"],
    "major_economies": ["United States", "China", "Japan", "Germany", "United Kingdom", "France"],
    "anglosphere": ["Australia", "United States", "Canada", "United Kingdom"],
    "eurozone_core": ["Germany", "France", "Italy", "Spain", "Netherlands", "Belgium"],
    "asian_tigers": ["South Korea", "Japan", "China"],
    "emerging_markets": ["China", "India", "Brazil", "Russia", "Mexico"],
}

MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median
COUNTER = 0
PW_COUNTER = 0

In [ ]:
def plot_contractions_complete(growth: pd.DataFrame) -> None:
    """Identify and plot quarterly GDP contractions."""
    
    if growth.empty:
        print("No data to analyze")
        return
    
    contractions = growth < 0
    contraction_count = contractions.sum(axis=1)

    start = pd.Period("2000Q1", freq="Q")
    title = "Number of Countries with Quarterly GDP Contraction (DB.nomics)"

    ax = contraction_count[contraction_count.index >= start].plot.bar()
    ax.set_xticks(ax.get_xticks()[::4])
    
    mg.finalise_plot(
        ax,
        title=title,
        ylabel="Count",
        rfooter=SOURCE,
        show=SHOW,
    )

    # Print nations in contraction
    if len(contractions) > 0:
        latest_contractions = contractions.iloc[-1][contractions.iloc[-1]]
        if len(latest_contractions) > 0:
            print(f"Latest countries in contraction (N={contraction_count.iloc[-1]}):")
            print(", ".join(latest_contractions.index.tolist()))
        else:
            print("No countries currently in GDP contraction")

def plot_recessions_complete(growth: pd.DataFrame) -> None:
    """Identify and plot technical recessions (two consecutive contractions)."""
    
    if growth.empty:
        print("No data to analyze")
        return
    
    recessions = (growth < 0) & (growth.shift(1) < 0)
    recession_count = recessions.sum(axis=1)

    title = "Number of Countries in Technical Recession (DB.nomics)"
    start = pd.Period("2000Q1", freq="Q")
    ax = recession_count[recession_count.index >= start].plot.bar()
    ax.set_xticks(ax.get_xticks()[::4])
    
    mg.finalise_plot(
        ax,
        title=title,
        ylabel="Count",
        rfooter=SOURCE,
        lfooter="Technical recession = two consecutive quarters of negative GDP growth",
        show=SHOW,
    )

    # Print nations in recession
    if len(recessions) > 0:
        latest_recessions = recessions.iloc[-1][recessions.iloc[-1]]
        if len(latest_recessions) > 0:
            print(f"Latest countries in technical recession (N={recession_count.iloc[-1]}):")
            print(", ".join(latest_recessions.index.tolist()))
        else:
            print("No countries currently in technical recession")

# Call the functions
if not gdp_complete.empty:
    plot_contractions_complete(gdp_complete)
    plot_recessions_complete(gdp_complete)

In [ ]:
def post_covid_analysis_complete(gdp_data: pd.DataFrame, start: str = "2022Q1") -> None:
    """Analyze GDP performance since COVID recovery."""
    
    if gdp_data.empty:
        print("No data for post-COVID analysis")
        return
    
    start_period = pd.Period(start, freq="Q")
    recent_gdp = gdp_data.loc[gdp_data.index >= start_period].dropna(how="all", axis=1)
    
    if recent_gdp.empty:
        print(f"No data available from {start}")
        return
    
    # Plot cumulative growth since start
    growth = ((1 + recent_gdp / 100).cumprod() - 1) * 100
    
    # Get latest data period for each country (like FRED notebook)
    final = growth.apply(pd.Series.last_valid_index).astype(str).str[2:]  
    gmap = {x: x + " " + str(y) for x, y in final.items()}
    
    cum_growth = {}
    for col in growth.columns:
        last_valid_idx = growth[col].last_valid_index()
        if last_valid_idx is not None:
            cum_growth[col] = growth[col].loc[last_valid_idx]
    
    if cum_growth:
        cum_growth_s = pd.Series(cum_growth).rename(index=gmap).sort_values()
        ax = cum_growth_s.plot.bar()
        ax.tick_params(axis="both", which="major", labelsize="x-small")
        
        mg.finalise_plot(
            ax,
            title=f"Cumulative GDP growth since {start_period-1} (DB.nomics)",
            ylabel="Per cent",
            rfooter=SOURCE,
            lfooter="To latest available data from international sources; see x-axis labels.",
            show=SHOW,
        )
    
    # Plot number of contractions since start - include ALL countries
    negative = recent_gdp < 0
    contraction_count = negative.sum().sort_values()
    
    if len(contraction_count) > 0:
        ax = contraction_count.plot.bar()
        ax.tick_params(axis="both", which="major", labelsize="x-small")
        
        mg.finalise_plot(
            ax,
            title=f"Number of negative GDP quarters since {start}",
            ylabel="Count",
            rfooter=SOURCE,
            show=SHOW,
        )

# Call the function
if not gdp_complete.empty:
    post_covid_analysis_complete(gdp_complete)

In [ ]:
# World context plots
def plot_world_gdp_complete(data: pd.DataFrame, focus_country: str = "Australia") -> None:
    """Plot focus country vs world GDP growth context."""
    
    if data.empty:
        print("No data to plot")
        return
    
    global PW_COUNTER
    PW_COUNTER += 1
    
    # Plot other countries as background
    other_countries = [col for col in data.columns if col != focus_country]
    if other_countries:
        background_data = data[other_countries]
        ax = mg.line_plot(background_data, width=0.3, color='blue', alpha=0.5, label_series=False)
    else:
        fig, ax = plt.subplots()
    
    # Plot mean and median if enough countries
    if len(other_countries) >= 3:
        mean = data[other_countries].mean(axis=1).where(
            data[other_countries].notna().sum(axis=1) >= len(other_countries) * MEAN_MEDIAN,
            other=np.nan,
        )
        median = data[other_countries].median(axis=1).where(
            data[other_countries].notna().sum(axis=1) >= len(other_countries) * MEAN_MEDIAN,
            other=np.nan,
        )
        
        mean.name = "Global mean"
        median.name = "Global median"
        
        mg.line_plot(mean, ax=ax, color="darkblue", style="--", width=2, label_series=True)
        mg.line_plot(median, ax=ax, color="darkred", style=":", width=2, label_series=True)
    
    # Highlight focus country if available
    if focus_country in data.columns:
        mg.line_plot(
            data[focus_country].dropna(),
            ax=ax,
            color="darkorange",
            width=3,
            label_series=True,
        )
    
    mg.finalise_plot(
        ax,
        title=f"{focus_country} quarterly GDP growth in world context (DB.nomics)",
        ylabel="Per cent per quarter",
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        lfooter=f"International sources via DB.nomics. Mean/median when >{int(MEAN_MEDIAN*100)}% report.",
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small", "ncol": 3},
        show=SHOW,
    )
    
if not gdp_complete.empty:
    print("\nRunning complete GDP analysis with DB.nomics data...")

    # Call world GDP plots
    plot_world_gdp_complete(gdp_complete, focus_country="Australia")
    
    # Recent years only
    recent_gdp = gdp_complete[gdp_complete.index.year >= 2020]
    plot_world_gdp_complete(recent_gdp, focus_country="Australia")
    
else:
    print("\n❌ No data available - analysis skipped")

## The End

In [ ]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda